In [1]:
import pandas as pd
import numpy as np

In [2]:
cols = (list(range(-1, 784)))
def prepare_ds(path, split=False, ratio=0.0):
    data = pd.read_csv(filepath_or_buffer=path, names=cols)
    data = data.sample(frac=1)
    if split == True:    
        d1X = np.array(data[:int(-data.shape[0] * ratio)][cols[1:]]).T
        d1Y = np.array(pd.get_dummies(data[:int(-data.shape[0] * ratio)][-1])[:]).T
        d2X = np.array(data[int(-data.shape[0] * ratio):][cols[1:]]).T
        d2Y = np.array(pd.get_dummies(data[int(-data.shape[0] * ratio):][-1])[:]).T
        return d1X, d1Y, d2X, d2Y
    if split == False:
        d1X = np.array(data[:][cols[1:]]).T
        d1Y = np.array(pd.get_dummies(data[:][-1])[:]).T
        return d1X, d1Y

In [3]:
data_path = '../digit-recognizer/mnist_train.csv'
test_path = '../digit-recognizer/mnist_test.csv'
train_X, train_Y, valid_X, valid_Y = prepare_ds(data_path, split=True, ratio=0.2)
test_X, test_Y, = prepare_ds(test_path)

In [4]:
m_train = train_X.shape[1]
m_test = valid_X.shape[1]
train_X.shape, train_Y.shape, valid_X.shape, valid_Y.shape

((784, 48000), (10, 48000), (784, 12000), (10, 12000))

In [5]:
def normalize(X, Y):
    X = (X - np.mean(X)) / (np.std(X))
    return X, Y

In [6]:
def compute_LogLoss(Yp, Yg):
    cost = -(Yg * np.log(Yp) + (1 - Yg) * np.log(1 - Yp))
    return np.mean(cost)

In [7]:
def forward(W, b, X, eval=False):
    Z = np.dot(W.T, X) + b
    A = 1. / (1 + np.exp(-Z))
    if eval == False:
        return A
    if eval == True:
        return np.argmax(A, axis=0)

In [16]:
def backward_mm(Yp, Yg, W, b, X, VdW_prev, Vdb_prev, learning_rate=0.1, beta=0.99):
    cost = compute_LogLoss(Yp.T, Yg.T)
    diff = Yp - Yg
    dW = np.dot(X, diff.T) / m_train
    db = np.sum(diff, axis=1, keepdims=True) / m_train

    VdW = beta * VdW_prev + (1 - beta) * dW
    Vdb = beta * Vdb_prev + (1 - beta) * db
    
    W -= VdW * learning_rate
    b -= Vdb * learning_rate

    return cost, W, b, VdW, Vdb

In [17]:
def accuracy(y_pred, y_actu):
    y_actt = np.argmax(y_actu, axis=0)
    return np.mean((y_pred == y_actt))

In [18]:
W = np.zeros((784, 10))
b = np.zeros((10, 1))
mini_batch = 6000
prep_X, prep_Y = normalize(train_X, train_Y)   
for i in range(1000):
    VdW_prev, Vdb_prev = 0, 0
    pred_Y = forward(W, b, prep_X)
    cost, W, b, VdW_prev, Vdb_prev = backward_mm(pred_Y, prep_Y, W, b, prep_X, VdW_prev, Vdb_prev)
    if i % 100 == 0:
        tr_X, tr_Y = prep_X, prep_Y
        vl_X, vl_Y = normalize(valid_X, valid_Y)
        tr_P = forward(W, b, tr_X, eval=True)
        vl_P = forward(W, b, vl_X, eval=True)

        print(i, cost, accuracy(tr_P, tr_Y), accuracy(vl_P, vl_Y))

0 0.6931471805599451 0.7339166666666667 0.7324166666666667
100 0.2616671214947002 0.771125 0.7725833333333333
200 0.21580583035937873 0.7968333333333333 0.8006666666666666
300 0.1896205296704058 0.813375 0.8169166666666666
400 0.172529692564997 0.824 0.8270833333333333
500 0.16038152595439673 0.831875 0.8353333333333334
600 0.15122281928166784 0.83775 0.8426666666666667
700 0.1440205213219646 0.8420416666666667 0.84675
800 0.13817616742811972 0.8462083333333333 0.8511666666666666
900 0.13331781600847795 0.8494791666666667 0.8549166666666667


In [23]:
W = np.zeros((784, 10))
b = np.zeros((10, 1))
prep_X, prep_Y = normalize(train_X, train_Y)
for i in range(1000):
    VdW_prev, Vdb_prev = 0, 0
    pred_Y = forward(W, b, prep_X)
    cost, W, b, VdW_prev, Vdb_prev = backward_mm(pred_Y, prep_Y, W, b, prep_X, VdW_prev, Vdb_prev)
    if i % 50 == 0:
        tr_X, tr_Y = prep_X, prep_Y
        vl_X, vl_Y = normalize(valid_X, valid_Y)
        tr_P = forward(W, b, tr_X, eval=True)
        vl_P = forward(W, b, vl_X, eval=True)

        print(i, cost, accuracy(tr_P, tr_Y), accuracy(vl_P, vl_Y))

0 0.6931471805599451 0.7318958333333333 0.7269166666666667
50 0.15959454754259883 0.8327708333333333 0.8339166666666666
100 0.1287349998293956 0.853125 0.8544166666666667
150 0.11492367172747289 0.863375 0.86375
200 0.10678538819475174 0.8687916666666666 0.8699166666666667
250 0.10131911374647845 0.8727291666666667 0.8733333333333333
300 0.09734946014925432 0.8762708333333333 0.877
350 0.09431184193487718 0.8791458333333333 0.8788333333333334
400 0.09189824256856557 0.8816041666666666 0.881
450 0.08992514441697294 0.88325 0.88275
500 0.08827585439193487 0.88475 0.8845
550 0.08687234520474586 0.8858541666666667 0.8858333333333334
600 0.08566030818120637 0.8871041666666667 0.8871666666666667
650 0.0846006828944108 0.8880625 0.8886666666666667
700 0.08366459716365303 0.8889583333333333 0.8893333333333333
750 0.08283020974759041 0.8898958333333333 0.8909166666666667
800 0.08208066672756485 0.8908541666666666 0.8915
850 0.08140273668896346 0.8917708333333333 0.8918333333333334
900 0.0807858

In [15]:
val_prep_X, val_prep_Y = normalize(valid_X, valid_Y)
valid_preds = forward(W, b, val_prep_X, eval=True)
accuracy(valid_preds, val_prep_Y)

0.8925833333333333

In [16]:
test_prep_X, test_prep_Y = normalize(test_X, test_Y)
test_preds = forward(W, b, test_prep_X, eval=True)
accuracy(test_preds, test_prep_Y)

0.9006